In [8]:

import torch
import torch.nn as nn
from torchvision import transforms, datasets
from torch.utils.data import Dataset, DataLoader
import os
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

data_transforms = transforms.Compose([
    transforms.RandomRotation(20),
    transforms.RandomAffine(degrees=0, translate=(0.2, 0.2)),
    transforms.RandomResizedCrop(size=224, scale=(0.8, 1.2)),
    transforms.RandomHorizontalFlip(p=1.0),
    transforms.ColorJitter(brightness=(0.8, 1.2)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])
base_dir = 'ikun/train'
dataset = datasets.ImageFolder(base_dir, transform=data_transforms)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)


class WXZ(nn.Module):
    def __init__(self, num_classes=2):
        super(WXZ, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 32, 3, 1, padding='same'),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, 1, padding='same'),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 128, 3, 1, padding='same'),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.ReLU(),
            nn.Linear(128 * 28 * 28, 1024),
            nn.ReLU(),
            nn.LayerNorm(1024),
            nn.Linear(1024, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x


model = WXZ().to(device)
loss_ikun = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
writer = SummaryWriter('ikun_CNN')
num_epochs = 50
model.train()
for epoch in range(num_epochs):
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_ikun(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        print(f'Epoch {epoch + 1}/{num_epochs}训练完成,loss:{running_loss / len(dataloader):.4f}, Acc: {100*correct/total:.2f}%')
    avg_loss = running_loss / len(dataloader)
    train_acc = 100 * correct / total
    writer.add_scalar('Train Loss', avg_loss, epoch)
    writer.add_scalar('Train Acc', train_acc, epoch)
    writer.add_images('train_imgs', inputs.cpu(), 10)
    writer.close()
test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])
test_dir = 'ikun/val'
test_dataset = datasets.ImageFolder(test_dir, transform=test_transform)
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False)
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
test_acc = 100 * correct / total
print(f'测试精准度:{test_acc:.2f}%')
writer.add_scalar('Test acc', test_acc, num_epochs)
writer.add_graph(model, torch.rand(1, 3, 224, 224).to(device))
writer.close()
torch.save(model.state_dict(), 'ikun_ball_model.pth')
writer.close()


Using device: cuda
Epoch 1/50训练完成,loss:0.0815, Acc: 50.00%
Epoch 1/50训练完成,loss:0.1464, Acc: 56.25%
Epoch 1/50训练完成,loss:0.2036, Acc: 58.33%
Epoch 1/50训练完成,loss:0.3040, Acc: 56.25%
Epoch 1/50训练完成,loss:0.3773, Acc: 55.00%
Epoch 1/50训练完成,loss:0.4592, Acc: 54.17%
Epoch 1/50训练完成,loss:0.5493, Acc: 53.57%
Epoch 1/50训练完成,loss:0.6516, Acc: 53.12%
Epoch 1/50训练完成,loss:0.7082, Acc: 55.56%
Epoch 1/50训练完成,loss:0.8301, Acc: 52.50%
Epoch 2/50训练完成,loss:0.0767, Acc: 62.50%
Epoch 2/50训练完成,loss:0.1347, Acc: 56.25%
Epoch 2/50训练完成,loss:0.1898, Acc: 58.33%
Epoch 2/50训练完成,loss:0.2428, Acc: 62.50%
Epoch 2/50训练完成,loss:0.3092, Acc: 67.50%
Epoch 2/50训练完成,loss:0.3851, Acc: 64.58%
Epoch 2/50训练完成,loss:0.4453, Acc: 64.29%
Epoch 2/50训练完成,loss:0.4935, Acc: 65.62%
Epoch 2/50训练完成,loss:0.5725, Acc: 65.28%
Epoch 2/50训练完成,loss:0.6015, Acc: 67.50%
Epoch 3/50训练完成,loss:0.0527, Acc: 87.50%
Epoch 3/50训练完成,loss:0.0766, Acc: 87.50%
Epoch 3/50训练完成,loss:0.0967, Acc: 87.50%
Epoch 3/50训练完成,loss:0.1420, Acc: 84.38%
Epoch 3/50训练完成,loss:0

In [14]:
import tkinter as tk
from tkinter import filedialog, Label, Button
from PIL import Image, ImageTk
import numpy as np

model1 = WXZ().to(device)
model1.load_state_dict(torch.load('ikun_ball_model.pth', map_location=device))
model1.eval()
trans = test_transform
root = tk.Tk()
root.title('篮球和鸡AI识别')
root.geometry('800x600')
title_label = Label(root, text='篮球和鸡识别系统', font=('Arial', 20))
title_label.pack(pady=20)

image_frame = tk.Frame(root)
image_frame.pack(pady=20)
image_label = Label(image_frame)
image_label.pack()

result_frame = tk.Frame(root)
result_frame.pack(pady=20)
result_label = Label(result_frame, text='等待导入图片', font=('Arial', 20))
result_label.pack()


def import_image():
    file_path = filedialog.askopenfilename(title='选择图片', filetypes=[('图片文件', '*.jpg *.png')])
    if file_path:
        img = Image.open(file_path).convert('RGB')
        img = img.resize((300, 300), Image.LANCZOS)
        img_tk = ImageTk.PhotoImage(img)
        image_label.config(image=img_tk)
        image_label.image = img_tk

        img_tensor = trans(img).unsqueeze(0).to(device)
        with torch.no_grad():
            outputs = model1(img_tensor)
            probs = torch.softmax(outputs, dim=1)
            _, predicted = torch.max(outputs, 1)
            ball_prob = probs[0, 0].item()
            chicken_prob = probs[0, 1].item()
            if predicted == 0:
                result = f'篮球的概率是{ball_prob:.4f}'
            else:
                result = f'鸡的概率是{chicken_prob:.4f}'
            result_label.config(text=result)


import_button = Button(root, text='导入图片', command=import_image, font=('Arial', 12), bg='#4CAF50', fg='white',padx=20, pady=10)
import_button.pack()

Instruction_label = Label(root, text='篮球还是鸡？AI快速出答案！', font=('Arial', 10), fg='#666')
Instruction_label.pack(pady=10)
root.mainloop()